# ETL - Fase 4: Consolidamos los df para alimentar las consultas
En esta seccion, consolidamos lo df generados en las fases anteriores de ETL. De igual manera, creamos dos subconjuntos de datos para alimentar los principales dos tipos de consulta: las funciones generales y el sistema de recomendacion (modelo ML).

In [1]:
import os
import pandas as pd

In [2]:
data_movies_subset = pd.read_csv(
    os.path.join("2_pipeline","data_movies_subset_limpia.csv"),index_col=0).convert_dtypes() # output paso 01
data_movies_normalizada = pd.read_csv(
    os.path.join("2_pipeline","data_movies_normalizada.csv"),index_col=0).convert_dtypes() # output paso 02
data_credits_normalizada = pd.read_csv(
    os.path.join("2_pipeline","credits_normalizada.csv"),index_col=0).convert_dtypes() # output paso 03

In [3]:
data_movies_subset.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 45346 entries, 0 to 45465
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   budget             45346 non-null  Int64  
 1   pelicula_id        45346 non-null  Int64  
 2   original_language  45335 non-null  string 
 3   overview           44405 non-null  string 
 4   popularity         45346 non-null  Float64
 5   release_date       45346 non-null  string 
 6   revenue            45346 non-null  Int64  
 7   runtime            45100 non-null  Int64  
 8   title              45346 non-null  string 
 9   vote_average       45346 non-null  Float64
 10  vote_count         45346 non-null  Int64  
 11  release_year       45346 non-null  Int64  
 12  return             45346 non-null  Float64
dtypes: Float64(3), Int64(6), string(4)
memory usage: 5.2 MB


In [4]:
data_movies_normalizada.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 55897 entries, 0 to 55896
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   pelicula_id               55897 non-null  Int64 
 1   franquicia_id             55897 non-null  Int64 
 2   franquicia                55897 non-null  string
 3   productora                53879 non-null  string
 4   productora_id             53879 non-null  Int64 
 5   genres_id                 55754 non-null  Int64 
 6   genres                    55754 non-null  string
 7   pais_isocode              55285 non-null  string
 8   pais_name                 55285 non-null  string
 9   spoken_languages_isocode  55728 non-null  string
 10  spoken_languages_name     55355 non-null  string
dtypes: Int64(4), string(7)
memory usage: 5.3 MB


In [5]:
data_credits_normalizada.info(verbose=True)
data_credits_normalizada.head(10)

<class 'pandas.core.frame.DataFrame'>
Index: 106173 entries, 0 to 106172
Data columns (total 6 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   pelicula_id           106173 non-null  Int64 
 1   prtgnst_name          106173 non-null  string
 2   prtgnst_gender_strng  106173 non-null  string
 3   prtgnst_nivel         106173 non-null  Int64 
 4   director              106138 non-null  string
 5   executive_producer    38468 non-null   string
dtypes: Int64(2), string(4)
memory usage: 5.9 MB


,pelicula_id,prtgnst_name,prtgnst_gender_strng,prtgnst_nivel,director,executive_producer
0,862,Tom Hanks,hombre,1,John Lasseter,Ed Catmull
1,862,Tom Hanks,hombre,1,John Lasseter,Steve Jobs
2,862,Tim Allen,hombre,2,John Lasseter,Ed Catmull
3,862,Tim Allen,hombre,2,John Lasseter,Steve Jobs
4,8844,Robin Williams,hombre,1,Joe Johnston,Larry J. Franco
5,8844,Robin Williams,hombre,1,Joe Johnston,Ted Field
6,8844,Robin Williams,hombre,1,Joe Johnston,Robert W. Cort
7,8844,Jonathan Hyde,hombre,2,Joe Johnston,Larry J. Franco
8,8844,Jonathan Hyde,hombre,2,Joe Johnston,Ted Field
9,8844,Jonathan Hyde,hombre,2,Joe Johnston,Robert W. Cort


## 4.1 Analisis de disponibilidad de datos


En esta seccion examinamos la disponibilidad de datos para nuestro universo de casos: peliculas. Conocer la disponibilidad de datos es indispensable por dos razones:

1. Determina la naturaleza de los joins: Un paso previo a la union de dfs es determinar cual de los dfs contiene la mayor cantidad de informacion disponible para nuestro universo de casos: peliculas. Esto determina la naturaleza del join; es decir, si hacemos un inner o un left join. En caso que un df contenga informacion sobre un mayor numero de casos que el otro, hacemos un left join con el fin garantizar el mayor numero de registros posible.

2. Afecta el desempeño de las consultas: diferencias en las disponibilidad de informacion puede afectar el desempeno de unas consultas con respecto a otras; es decir, mientras que algunas consultas pueden gozar de una gran disponibilidad de informacion, otras pueden verse seriamente afectadas por informacion limitada. Conocer esta ventajas y limitaciones nos permite anticipar futuros problemas en el desempeno de las consultas.

### 4.1.1 Descripcion de disponibilidad de informacion

In [6]:
# evaluamos cual df contiene informacion para el mayor numero de casos.
movies_subset_peliculas=data_movies_subset['pelicula_id'].nunique()
movies_normalizada_peliculas=data_movies_normalizada['pelicula_id'].nunique()
credits_normalizada_peliculas=data_credits_normalizada['pelicula_id'].nunique()
print("Universo de pelicula unicas, segun df de referencia:",movies_subset_peliculas)
print("Universo de pelicula unicas, segun movies_normalizada:",movies_normalizada_peliculas)
print("Universo de pelicula unicas, segun credits_normalizada:",credits_normalizada_peliculas)
print("Porcentaje de valores nulos en columnas de movies_normalizada del total de registros (df referencia):",
    round((1-(movies_normalizada_peliculas/movies_subset_peliculas))*100,2),"%")
print("Porcentaje de valores nulos en columnas de credits_normalizada del total de registros (df referencia):",
    round((1-(credits_normalizada_peliculas/movies_subset_peliculas))*100,2),"%")

Universo de pelicula unicas, segun df de referencia: 45346
Universo de pelicula unicas, segun movies_normalizada: 4487
Universo de pelicula unicas, segun credits_normalizada: 42317
Porcentaje de valores nulos en columnas de movies_normalizada del total de registros (df referencia): 90.1 %
Porcentaje de valores nulos en columnas de credits_normalizada del total de registros (df referencia): 6.68 %


Con base en lo anterior, la base de datos final tendria la siguientes caracteristicas:
Todos las peliculas tendria valores nulos en alguna columna.
Las columnas con mayor numero de valores nulos serian las columnas que corresponden al df `data_movies_normalizada`.

### 4.1.2 Implicaciones de disponibilidad de datos para las consultas

Las funciones que mas se afectarian por esta baja disponibilidad de datos seria las siguientes consultas:

- CUALES?????

Lo anterior debe orientar nuestro diseno del modelo de ML. Un modelo que haga uso de las columnas que corresponden a data_movies_normalizada (i.e., franquicia, productora, el genero de la pelicula (genres), e idiomas hablados (spoken_languages_name)), estaria desaprovechando la disponbilidad de datos, puesto que solo tendria informacion para el ~10% de universo de casos en nuestra base de dato final.

Por tal motivo, lo mejor seria hacer uso la informacion contenida en data_movies_subset y en data_credits_normalizada al diseñar el modelo de ML.

## 4.2 Generacion de dfs para alimentar las consultas

In [7]:
# exportamos el df final
data_mvp_final_normalizada = pd.merge(
    pd.merge(data_movies_subset, # tomamos como principal referencia el df con el mayor numero de registros
    data_movies_normalizada,
    how='left',
    on=['pelicula_id']),
    data_credits_normalizada, # tomamos como ultima referencia el df con el menor numero de registros
    how='left',
    on=['pelicula_id'])
data_mvp_final_normalizada.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 310040 entries, 0 to 310039
Data columns (total 28 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   budget                    310040 non-null  Int64  
 1   pelicula_id               310040 non-null  Int64  
 2   original_language         310024 non-null  string 
 3   overview                  308284 non-null  string 
 4   popularity                310040 non-null  Float64
 5   release_date              310040 non-null  string 
 6   revenue                   310040 non-null  Int64  
 7   runtime                   309672 non-null  Int64  
 8   title                     310040 non-null  string 
 9   vote_average              310040 non-null  Float64
 10  vote_count                310040 non-null  Int64  
 11  release_year              310040 non-null  Int64  
 12  return                    310040 non-null  Float64
 13  franquicia_id             214036 non-null  I

Separamos el df final en dos subconjuntos:
1. df para alimentar las funciones
2. df para alimentar el modelo ml y el sistema de recomendacion.

### 4.2.1. Subconjunto de datos para alimentar las funciones

In [8]:
# primer subconjunto: df para alimentar las funciones
columns_funciones=[
    'pelicula_id', # todas
    'title', # varias
    'original_language', # def peliculas_idioma( Idioma: str )
    'runtime', # def peliculas_duracion( Pelicula: str )
    'release_date', # def peliculas_duracion( Pelicula: str )
    'franquicia', # def franquicia( Franquicia: str )
    'budget', # def franquicia( Franquicia: str )
    'revenue', # def franquicia( Franquicia: str )
    'pais_name', # def peliculas_pais( Pais: str )
    'productora', # productoras_exitosas( Productora: str ),
    'director', # get_director( nombre_director )
    'return', # get_director( nombre_director )
    'release_year' # get_director( nombre_director )
    ]

In [9]:
data_mvp_funciones=data_mvp_final_normalizada.loc[:,columns_funciones].drop_duplicates()
print("Universo de peliculas unicas en el df que alimenta las funciones:",
      data_mvp_funciones['pelicula_id'].nunique())

Universo de peliculas unicas en el df que alimenta las funciones: 45346


In [10]:
# finalmente, calculamos el retorno por director
data_mvp_funciones_exitodir=data_mvp_funciones[['director','revenue','budget']].loc[
    (data_mvp_funciones['revenue']!=0) & (data_mvp_funciones['budget']!=0) # filtramos unicamente aquellas filas para las que hay informacion
]
print("Numero de directores con informacion disponible", data_mvp_funciones_exitodir['director'].nunique())
data_exitodir_revenue=data_mvp_funciones_exitodir[['director','revenue']].groupby(['director']).sum()
data_exitodir_budget=data_mvp_funciones_exitodir[['director','budget']].groupby(['director']).sum()
data_mvp_funciones_exitodir=pd.merge(
    data_exitodir_revenue,
    data_exitodir_budget,
    on=['director'])
data_mvp_funciones_exitodir['return']=data_mvp_funciones_exitodir['revenue']/data_mvp_funciones_exitodir['budget']
data_mvp_funciones_exitodir.rename(columns={'return': 'director_return'},inplace=True)
data_mvp_funciones_exitodir.drop(columns=['revenue','budget'],inplace=True)
data_mvp_funciones_exitodir.reset_index(inplace=True)
data_mvp_funciones_exitodir.info()

Numero de directores con informacion disponible 2616
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2616 entries, 0 to 2615
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   director         2616 non-null   string 
 1   director_return  2616 non-null   Float64
dtypes: Float64(1), string(1)
memory usage: 43.6 KB


In [11]:
data_mvp_funciones=data_mvp_funciones.merge(
    data_mvp_funciones_exitodir,
    on=['director']
)
data_mvp_funciones.to_csv(os.path.join("3_output","data_mvp_final_funciones.csv"))
data_mvp_funciones.info()
data_mvp_funciones.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22166 entries, 0 to 22165
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   pelicula_id        22166 non-null  Int64  
 1   title              22166 non-null  string 
 2   original_language  22166 non-null  string 
 3   runtime            22157 non-null  Int64  
 4   release_date       22166 non-null  string 
 5   franquicia         8903 non-null   string 
 6   budget             22166 non-null  Int64  
 7   revenue            22166 non-null  Int64  
 8   pais_name          8850 non-null   string 
 9   productora         8755 non-null   string 
 10  director           22166 non-null  string 
 11  return             22166 non-null  Float64
 12  release_year       22166 non-null  Int64  
 13  director_return    22166 non-null  Float64
dtypes: Float64(2), Int64(5), string(7)
memory usage: 2.5 MB


pelicula_id              0
title                    0
original_language        0
runtime                  9
release_date             0
franquicia           13263
budget                   0
revenue                  0
pais_name            13316
productora           13411
director                 0
return                   0
release_year             0
director_return          0
dtype: int64

### 4.2.2. Subconjunto de datos para alimentar el modelo ML

In [12]:
# segundo subconjunto: df para alimentar el modelo de ML
columns_ml=[
    'pelicula_id',
    'title',
    'popularity',
    'vote_average',
    'release_year',
    'franquicia',
    'original_language',
    'prtgnst_name',
    'prtgnst_gender_strng',
    'prtgnst_nivel',
    'director',
    'executive_producer'
    ]

In [13]:
# subconjunto de datos para modelo ml
data_mvp_ml=data_mvp_final_normalizada.loc[:,columns_ml].drop_duplicates()
print("Universo de peliculas unicas en el df que alimenta el modelo ml:",
      data_mvp_funciones['pelicula_id'].nunique())

Universo de peliculas unicas en el df que alimenta el modelo ml: 14740


#### 4.2.2.1. Imputacion de valores nulos

In [14]:
# identificamos las columnas con nulos
data_mvp_ml.info()
data_mvp_ml.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 109186 entries, 0 to 310039
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   pelicula_id           109186 non-null  Int64  
 1   title                 109186 non-null  string 
 2   popularity            109186 non-null  Float64
 3   vote_average          109186 non-null  Float64
 4   release_year          109186 non-null  Int64  
 5   franquicia            13182 non-null   string 
 6   original_language     109170 non-null  string 
 7   prtgnst_name          106143 non-null  string 
 8   prtgnst_gender_strng  106143 non-null  string 
 9   prtgnst_nivel         106143 non-null  Int64  
 10  director              106108 non-null  string 
 11  executive_producer    38458 non-null   string 
dtypes: Float64(2), Int64(3), string(7)
memory usage: 11.3 MB


pelicula_id                 0
title                       0
popularity                  0
vote_average                0
release_year                0
franquicia              96004
original_language          16
prtgnst_name             3043
prtgnst_gender_strng     3043
prtgnst_nivel            3043
director                 3078
executive_producer      70728
dtype: int64

In [15]:
# Evaluamos la cantidad de peliculas sin idioma
print("Numero de peliculas sin idioma:",data_mvp_ml.loc[data_mvp_ml['original_language'].isna()==True]['pelicula_id'].nunique())

# guardamos la posicion de las filas para comparar el resultado de la imputacion
language_null_index=data_mvp_ml.loc[data_mvp_ml['original_language'].isna()==True].index

# visualizamos
data_mvp_ml.loc[data_mvp_ml['original_language'].isna()==True].head(3)


Numero de peliculas sin idioma: 11


,pelicula_id,title,popularity,vote_average,release_year,franquicia,original_language,prtgnst_name,prtgnst_gender_strng,prtgnst_nivel,director,executive_producer
194537,283101,Shadowing the Third Man,0.017007,0.0,2004,<NA>,<NA>,John Hurt,hombre,2,Frederick Baker,<NA>
212270,103902,Unfinished Sky,0.359818,6.4,2007,<NA>,<NA>,William McInnes,hombre,1,Peter Duncan,<NA>
212271,103902,Unfinished Sky,0.359818,6.4,2007,<NA>,<NA>,Monic Hendrickx,mujer,2,Peter Duncan,<NA>


In [16]:
# Imputamos en idioma con base en el titulo de la pelicula
from langdetect import detect, DetectorFactory
DetectorFactory.seed = 0
print("Ejemplo de identificacion de idioma para 'Lettre d'une inconnue':",detect("Lettre d'une inconnue"))

language_fill = data_mvp_ml.loc[
    data_mvp_ml['original_language'].isna()==True, 'title'].apply(detect) # aplicamos la funcion de identificacion de idiomas a los casos nulos
data_mvp_ml.loc[
    data_mvp_ml['original_language'].isnull() & (data_mvp_ml['original_language'].isna()==True), # condiciones
    'original_language'] = language_fill # imputamos los valores al base de datos

# visualizamos los cambios realizados
data_mvp_ml[['pelicula_id','title','original_language']].loc[language_null_index] 

Ejemplo de identificacion de idioma para 'Lettre d'une inconnue': fr


,pelicula_id,title,original_language
194537,283101,Shadowing the Third Man,en
212270,103902,Unfinished Sky,en
212271,103902,Unfinished Sky,en
219025,359195,13 Fighting Men,en
269410,147050,Lambchops,ca
269411,147050,Lambchops,ca
284685,257095,Prince Bayaya,tr
296700,332742,Song of Lahore,en
299040,144410,Annabelle Serpentine Dance,fr
299041,144410,Annabelle Serpentine Dance,fr


In [17]:
# asumimos que los valores nulos en franquicia corresponden a peliculas que no pertnenecen a ninguna franquicia.
# Con base en lo anterior, imputamos el valor 'Sin franquicia' para los casos nulos
data_mvp_ml['franquicia'].fillna('Sin franquicia', inplace = True)

In [18]:
# Nuestros resultados despues de la primera fase de imputacion
data_mvp_ml.info()
data_mvp_ml.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 109186 entries, 0 to 310039
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   pelicula_id           109186 non-null  Int64  
 1   title                 109186 non-null  string 
 2   popularity            109186 non-null  Float64
 3   vote_average          109186 non-null  Float64
 4   release_year          109186 non-null  Int64  
 5   franquicia            109186 non-null  string 
 6   original_language     109186 non-null  string 
 7   prtgnst_name          106143 non-null  string 
 8   prtgnst_gender_strng  106143 non-null  string 
 9   prtgnst_nivel         106143 non-null  Int64  
 10  director              106108 non-null  string 
 11  executive_producer    38458 non-null   string 
dtypes: Float64(2), Int64(3), string(7)
memory usage: 15.4 MB


pelicula_id                 0
title                       0
popularity                  0
vote_average                0
release_year                0
franquicia                  0
original_language           0
prtgnst_name             3043
prtgnst_gender_strng     3043
prtgnst_nivel            3043
director                 3078
executive_producer      70728
dtype: int64

#### 4.2.2.2. Imputacion de valores y creacion del grupo 'peliculas de baja circulacion'

Nos concentramos en las columnas restantes que contienen nulos.

In [19]:
columns_grupo_bjcrclcn=[
    'prtgnst_name',
    'prtgnst_gender_strng',
    'prtgnst_nivel',
    'director',
    'executive_producer']
data_mvp_ml[columns_grupo_bjcrclcn].isna().sum()

prtgnst_name             3043
prtgnst_gender_strng     3043
prtgnst_nivel            3043
director                 3078
executive_producer      70728
dtype: int64

Consideramos las peliculas que contienen valores nulos en estas columnas pertenecen a un grupo: peliculas de baja circulacion. El website the IMDB señala que su fuente de datos no es unicamente estudios de produccion y realizadores de cine, sino que "[...] la mayor parte de nuestra información procede de profesionales y visitantes" ([IMDB](https://help.imdb.com/article/imdb/general-information/where-does-the-information-on-imdb-come-from/GGD7NGF5X3ECFKNN?ref_=helpart_nav_17#)). Por tanto, consideramos que si una pelicula no contiene informacion en la base de datos IMDB es por que pertenece a un grupo que denominamos "peliculas de baja circulacion". Por tal motivo, imputamos el valor `baja_circulacion` en los casos `NA` para asociar la membresia de una pelicula con dicho grupo.

A su vez, la membresia al grupo "peliculas de baja circulacion" ayuda a capturar diferentes intensidades de membresia. Por ejemplo, una pelicula podria tener valores nulos en solo un par de columnas, mientras que otra pelicula podria tener valores nulos en varias columnas; la interseccion de `baja_circulacion` en dos columnas señalaria un caso de bajo grado de "poca circulacion", mientras que la interseccion de varias columnas con el valor `baja_circulacion` señalaria un alto grado de membresia al grupo "peliculas de baja circulacion". De esta manera, incorporamos a nuestro modelo ML no solo la membresia al grupo "peliculas de baja circulacion" sino tambien diferentes intensidades de membresia, permitiendo la identificacion de clusters que de otra manera (p. ej., eliminando los valores nulos) no seria posible.

In [20]:
# algunos casos de peliculas que pertenecen al grupo "peliculas con baja circulacion"
baja_circulacion_index=data_mvp_ml.loc[data_mvp_ml['director'].isna()==True].index # guardamos para comparar
data_mvp_ml.loc[data_mvp_ml['director'].isna()==True].head(5)


,pelicula_id,title,popularity,vote_average,release_year,franquicia,original_language,prtgnst_name,prtgnst_gender_strng,prtgnst_nivel,director,executive_producer
914,124639,Target,0.001205,0.0,1995,Sin franquicia,en,<NA>,<NA>,<NA>,<NA>,<NA>
1648,56088,The Show,0.159596,5.3,1995,Sin franquicia,en,<NA>,<NA>,<NA>,<NA>,<NA>
1947,43475,Gumby: The Movie,0.090452,5.0,1995,Sin franquicia,en,<NA>,<NA>,<NA>,<NA>,<NA>
3164,42981,Frank and Ollie,0.142357,7.5,1995,Sin franquicia,en,<NA>,<NA>,<NA>,<NA>,<NA>
3429,24257,The Endless Summer 2,0.408159,7.1,1994,The Endless Summer Collection,en,<NA>,<NA>,<NA>,<NA>,<NA>


In [21]:
# inputamos valores nulos con las columnas seleccionadas con el valor 'baja_circulacion';
# cambiamos prtgnst_nivel a string para facilitar a imputacion de valores
data_mvp_ml['prtgnst_nivel']=data_mvp_ml['prtgnst_nivel'].astype('string')
data_mvp_ml[columns_grupo_bjcrclcn]=data_mvp_ml[columns_grupo_bjcrclcn].fillna('baja_circulacion')
data_mvp_ml.loc[baja_circulacion_index].head(5)

,pelicula_id,title,popularity,vote_average,release_year,franquicia,original_language,prtgnst_name,prtgnst_gender_strng,prtgnst_nivel,director,executive_producer
914,124639,Target,0.001205,0.0,1995,Sin franquicia,en,baja_circulacion,baja_circulacion,baja_circulacion,baja_circulacion,baja_circulacion
1648,56088,The Show,0.159596,5.3,1995,Sin franquicia,en,baja_circulacion,baja_circulacion,baja_circulacion,baja_circulacion,baja_circulacion
1947,43475,Gumby: The Movie,0.090452,5.0,1995,Sin franquicia,en,baja_circulacion,baja_circulacion,baja_circulacion,baja_circulacion,baja_circulacion
3164,42981,Frank and Ollie,0.142357,7.5,1995,Sin franquicia,en,baja_circulacion,baja_circulacion,baja_circulacion,baja_circulacion,baja_circulacion
3429,24257,The Endless Summer 2,0.408159,7.1,1994,The Endless Summer Collection,en,baja_circulacion,baja_circulacion,baja_circulacion,baja_circulacion,baja_circulacion


In [22]:
# El resultado despues de completar todas las fases de imputacion: exportamos el df para alimentar el modelo ML
data_mvp_ml.to_csv(os.path.join("3_output","data_mvp_final_ml.csv"))
data_mvp_ml.info()
data_mvp_ml.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 109186 entries, 0 to 310039
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   pelicula_id           109186 non-null  Int64  
 1   title                 109186 non-null  string 
 2   popularity            109186 non-null  Float64
 3   vote_average          109186 non-null  Float64
 4   release_year          109186 non-null  Int64  
 5   franquicia            109186 non-null  string 
 6   original_language     109186 non-null  string 
 7   prtgnst_name          109186 non-null  string 
 8   prtgnst_gender_strng  109186 non-null  string 
 9   prtgnst_nivel         109186 non-null  string 
 10  director              109186 non-null  string 
 11  executive_producer    109186 non-null  string 
dtypes: Float64(2), Int64(2), string(8)
memory usage: 15.3 MB


pelicula_id             0
title                   0
popularity              0
vote_average            0
release_year            0
franquicia              0
original_language       0
prtgnst_name            0
prtgnst_gender_strng    0
prtgnst_nivel           0
director                0
executive_producer      0
dtype: int64